## Purpose - To scrape data on authors, their affiliation and reported addresses at the level of each paper

### Import Packages

In [1]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException
import pickle

In [ ]:
EXE_PATH = r"C:\Users\nik596\Downloads\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

In [ ]:
base = "https://www.nber.org/papers?page=1&perPage=100&sortBy=public_date"
browser.get(base)
browser.implicitly_wait(10)

text = browser.find_element_by_class_name("search__info-results").text
pages = int(text[text.find('F') + 2 : text.find('R') - 1]) + 1
pages = math.ceil(pages/100)
pages

In [ ]:
paper_links = []
url = "https://www.nber.org/papers?page={q}&perPage=100&sortBy=public_date"

for i in range(1, pages + 1, 1):
    
    if i%10 ==0:
        print(i)
    
    browser.get(url.format(q = str(i)))
    browser.implicitly_wait(30)
    
    
    for element in browser.find_elements_by_xpath('//a[contains(@href,"/papers/")]'):
        paper_link = element.get_attribute('href')
        paper_links.append(paper_link)

In [ ]:
paper_links = set(paper_links)
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','wb') as f:
   pickle.dump(paper_links, f)

In [2]:
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','rb') as f:
   paper_links = pickle.load(f)

In [3]:
scrape_all =0
if scrape_all:
    papers = []
    scraped= []
    dic = {}
else:
    with open(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.txt','rb') as f:
       papers = pickle.load(f)
    df=pd.DataFrame.from_dict(papers).T
    scraped = list(df['paper_link'])
    print("Scraped", len(scraped))
    dic = papers
print("Total", len(paper_links))

Scraped 27816
Total 30487


In [4]:
count = len(papers)

errors = []
paper_links = [link for link in paper_links if link not in scraped]
print("Left", len(paper_links))
for paper_link in paper_links:
        
        try:
            html = urlopen(paper_link)
            soup = BeautifulSoup(html, 'lxml')
        
            paper_name = soup.find('h1', class_ = 'page-header__title').text.strip()
        
            if paper_link not in scraped:
                #print(paper_link)
                dic[count] = {'paper': paper_name}
                #print(paper_name)
                dic[count]['paper_link'] = paper_link
                
                paper_month = soup.find('time').text
                dic[count]['paper month'] = paper_month
                
                temp = soup.find('div', class_ = 'page-header__citation-item').text.strip()
                dic[count]['paper number'] = temp[temp.find('Paper') + 6:]
                
                abstract = soup.find('div', class_ = 'page-header__intro-inner').text.strip()
                dic[count]['paper abstract'] = abstract
                
                for each in soup.find_all('div', class_ = 'info-grid__item'):
                    text = [t.text for t in each.find_all('a')]
                    l = len(text)
    
                    if 'topics' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper topic ' + str(c + 1)] = text[c]
                    elif 'programs' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper program ' + str(c + 1)] = text[c]
                    elif 'working groups' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper group ' + str(c + 1)] = text[c]
                    elif 'projects' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper project ' + str(c + 1)] = text[c]
        
                authors = soup.find_all('div', class_ = 'page-header__authors')[0].find_all('a')
        
                num = 1
                for author in authors:
            
                    author_name = author.text
                    author_link = author.get('href')
                    dic[count]['author ' + str(num)] = author_name
                    dic[count]['author link ' + str(num)] = author_link
            
                    author_link = "https://www.nber.org" + author_link
                    html = urlopen(author_link)
                    soup = BeautifulSoup(html, 'lxml')
                    
                    pos = soup.find_all('div', class_ = 'person-header__title')[0]
                    if len(pos) != 0:
                        author_position = pos.text.strip()
                        dic[count]['author position ' + str(num)] = author_position
                    else:
                        dic[count]['author position ' + str(num)] = ""
                        
                    affil = soup.find_all('div', class_ = 'person-header__summary')
                    if len(affil) != 0:
                        author_affiliation = affil[0].text.strip()
                        dic[count]['author affiliation ' + str(num)] = author_affiliation
                    else:
                        dic[count]['author affiliation ' + str(num)] = ""
                        
                    add = soup.find_all('div', 'contact-card__social')
                    if len(add) != 0:
                        author_address = add[-1].text.strip().replace('\n', ' ')
                        dic[count]['author address ' + str(num)] = author_address
                    else:
                        dic[count]['author address ' + str(num)] = ""
            
                    num+=1
                #print(count)
                count += 1
                if count%50 ==0:
                    print(count)
                    print(paper_name)
                    with open(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.txt','wb') as f:
                       pickle.dump(dic, f)
        except:
            errors.append(paper_link)
            print(errors)

Left 2671
27850
International VAT Harmonization: Economic Effects
27900
Patent Protection: Of What Value and for How Long?
27950
Nonlinear Programming Method for Dynamic Programming
28000
Country Risk and the Organization of International Capital Transfer
28050
Representing Symmetric Rank Two Updates
28100
Lump-Sums, Profit Sharing, and the Labor Costs in the Union Sector
28150
The Pension Cost of Changing Jobs
28200
The Economic Theory of Illegal Goods: The Case of Drugs
28250
Two Hundred Years of Health and Medical Care: The Importance of Medical Care for Life Expectancy Gains
28300
Expected Fiscal Policy and the Recession of 1982
28350
Addressing the Transfer-Pricing Problem in an Origin-Basis X Tax
28400
The Determinants of Income Tax Compliance: Evidence from a Controlled Experiment in Minnesota
28450
Price and Output Adjustment in Japanese Manufacturing
28500
The Timing and Spacing of Births and Women's Labor Force Participation: An Economic Analysis
28550
Corporate Supply of Ind

In [5]:
df=pd.DataFrame.from_dict(dic).T
df.to_excel(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.xlsx',engine='xlsxwriter')

In [6]:
df

,paper,paper_link,paper month,paper number,paper abstract,paper topic 1,paper topic 2,paper topic 3,paper topic 4,paper topic 5,...,author affiliation 26,author address 26,author 27,author link 27,author position 27,author affiliation 27,author address 27,paper group 7,paper group 8,paper group 9
0,Equilibrium Health Spending and Population Agi...,https://www.nber.org/papers/w19856,January 2014,19856,The apparently unrelenting growth in the GDP-s...,"Health, Education, and Welfare",Health,Labor Economics,Demography and Aging,Labor Supply and Demand,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wages and Prices Are Not Always Sticky: A Cent...,https://www.nber.org/papers/w0847,January 1982,0847,"Arthur M. Okun's last book, Prices and Quantit...",Macroeconomics,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,How Does Social Security Reform Indecision Aff...,https://www.nber.org/papers/w28850,May 2021,28850,The Social Security trust fund will be exhaust...,Financial Economics,Public Economics,National Fiscal Issues,Labor Economics,Labor Supply and Demand,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Global Business Cycles and Credit Risk,https://www.nber.org/papers/w11493,July 2005,11493,The potential for portfolio diversification is...,Econometrics,Estimation Methods,Macroeconomics,Macroeconomic Models,Financial Economics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Supplemental Security Income (SSI) Program,https://www.nber.org/papers/w21209,May 2015,21209,The SSI program provides cash assistance to so...,Public Economics,National Fiscal Issues,Labor Economics,Demography and Aging,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30482,The African Growth Miracle,https://www.nber.org/papers/w18490,October 2012,18490,Measures of real consumption based upon the ow...,Development and Growth,Growth and Productivity,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30483,Are All Banking Crises Alike? The Japanese Ex...,https://www.nber.org/papers/w7253,July 1999,7253,This paper examines episodes of banking sector...,Macroeconomics,Money and Interest Rates,Financial Economics,Financial Institutions,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30484,Police Force Size and Civilian Race,https://www.nber.org/papers/w28202,December 2020,28202,We report the first empirical estimate of the ...,Subnational Fiscal Issues,Labor Economics,Demography and Aging,Other,Law and Economics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30485,A Theory of the Production and Allocation of E...,https://www.nber.org/papers/w0184,July 1977,0184,The purpose of this paper is to analyze system...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
